In [1]:
import pandas as pd
import numpy as np
import os
from ts_data import TS_Data
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as F
from torchvision import transforms, utils
import torch.nn as nn
from CNN import CNN
from metrics import nll

ImportError: cannot import name 'fde' from 'metrics' (/home/warren/Documents/UGRA/AIM/metrics.py)

In [2]:
fdir = "/home/warren/Documents/UGRA/AIM/csv_files"
def get_dataset(fdir, in_len,pred_len):
    l = os.listdir(fdir)
    datasets = [None]*len(l)
    for i in range(len(l)):
        temp = os.path.join(fdir,l[i])
        datasets[i] = TS_Data(temp, in_len,pred_len)
    dataset =  torch.utils.data.ConcatDataset(datasets)
    return dataset


In [3]:
inlen = 8
predlen=8
full_dataset = get_dataset(fdir,inlen, predlen)
train_len = int(.8* len(full_dataset))
test_len = len(full_dataset)-train_len
train_data, test_data = torch.utils.data.random_split(full_dataset, [train_len, test_len])
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64)

## Training 

In [4]:
#model = CNN()
#take out the z 

In [5]:
#model = nn.LSTM(2, 5,5, batch_first=True) #ensure that batch is first 
model = CNN(8,4)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = nll()
optimizer = torch.optim.Adam(model.parameters())
epochs = 4
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [6]:
    #list of predictions is [batch size, steps - 8, ]    
def pred_to_samples(num_samples, pred):
        sx = torch.exp(pred[2]) #sx
        sy = torch.exp(pred[3]) #sy
        corr = torch.tanh(pred[4]) #corr
        sxsy = sx*sy
        cov = torch.FloatTensor([[sx*sx, corr*sxsy],[corr*sxsy, sy*sy]])
        means = torch.FloatTensor([pred[0],pred[1]])
        dist = torch.distributions.multivariate_normal.MultivariateNormal(means, cov)
        return dist.sample((num_samples,))
#no idea if I need to detach for these 

def ade_dist(pred,target):
        return (torch.sum(torch.square(pred-target))/pred.shape[1]).item() #assume the middle dimension


def fde_dist(pred, target):
        return (torch.sum(torch.square(pred[-1] - target[-1]))).item() #only compare the final location of pred to final location of tgt

In [7]:
for e in range(epochs):
    for idx, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds, (h_0,c_0)= model(inputs)
        loss  = criterion(preds, labels)
        
        loss.backward()
        optimizer.step()
        
        
    #Validation 
    with torch.no_grad():
        batch_sum_ade = 0
        batch_sum_fde=0
        for idx, (inputs, labels) in enumerate(test_dataloader): #batch size, steps, etc
            inputs = inputs.to(device)
            labels = labels.to(device)
            T = inputs.shape[0]
            N = inputs.shape[1]
            preds, (h,c) = model(inputs) #each step has a distribution. So take a sample of each step and then turn it into something cumulative

            for b in range(T): #iterate over batches
                curr = preds[b,:,:]
                #iterate over curr
                num_samples = 20
                num_steps = curr.shape[0]
                sample_list = []*num_steps
                for i in range(num_steps):
                    sample_list[i] = pred_to_samples(num_samples, curr[i])
                    #now should have a list of samples for each step

                samples = torch.stack(samples,dim=1) #stack it along middle dim to get [num_samples, num_steps, (x,y)]
                #now cumsum it 
                abs_samples = samples.cumsum(dim=1) #dim =1 since it is the number of steps shape is [num_samples, steps, rel pos]
                # now take the minimum ade/fde of the absolute samples to it 
                fde_dists = []
                ade_dists = []
                for i in range(num_samples):
                    #calculate the metrics for each
                    fde_dists.append(fde_dist(abs_samples[i]))
                    ade_dists.append(ade_dist(abs_samples[i]))
                #now take the min
                min_fde = min(fde_dists)
                min_ade = min(ade_dists)
                batch_sum_ade+=min_ade
                batch_sum_fde+=min_fde
            
            
        batch_sum_ade/= len(test_dataloader)
        batch_sum_fde/= len(test_dataloader)
            
    

/home/warren/mlenv/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


ValueError: too many values to unpack (expected 2)

# Issues
1. Had to change prediction length to the same as the input length for LSTM
2. Do i need to detach or is calling item() sufficient
3. The [batch_size, 8,5] means that each step has a distribution. So each x,y in a sequence of steps is from a different distribution and then make absolute
4. Making a CNN compatible with time series 

Assume take in 8 time steps, and predict 4 steps
1. Input into model, and model will return distribution parameters
2. Take a loss with 4 time steps and compare to distribution
3. 

1. Model predicts distribution, parameters, mux,muy sx,sy, rho
2. creat mean and cov matrix to make distribution. Torch multivariate_norm. tanh for coeff, exp for sx 
3. generate samples from distribution, 20 trajecctories . NOTE: each step has its own distribution. So for one trajectory, use 8 distributions
4. Convert samples from relative to absolute
4. Compare samples to the target one, take the minimum 




Issue with the prediction length, set it to the same as the input length. Changed the [:,1] to [...,1] etc

In [ ]:
for b in range(num_batches):
    curr = preds[b,:,:]
    #iterate over curr
    num_samples = 20
    num_steps = curr.shape[0]
    sample_list = []*num_steps
    for i in range(num_steps):
        sample_list[i] = pred_to_samples(num_samples, curr[i])
        #now should have a list of samples for each step
        
    samples = torch.stack(samples,dim=1) #stack it along middle dim to get [num_samples, num_steps, (x,y)]
    #now cumsum it 
    abs_samples = samples.cumsum(dim=1) #dim =1 since it is the number of steps shape is [num_samples, steps, rel pos]
    # now take the minimum ade/fde of the absolute samples to it 
    fde_dists = []
    ade_dists = []
    for i in range(num_samples):
        #calculate the metrics for each
        fde_dists.append(fde_dist(abs_samples[i]))
        ade_dists.append(ade_dist(abs_samples[i]))
    #now take the min
    min_fde = min(fde_dists)
    min_ade = min(ade_dists)
    
        

In [ ]:
a = torch.rand((5,4))

In [ ]:
dist = torch.distributions.multivariate_normal.MultivariateNormal(torch.FloatTensor([0,0]), torch.eye(2))
samples = dist.sample((20,8))
samples.shape

In [ ]:
samples[0]

In [ ]:
s1 = pred_to_samples(20,torch.FloatTensor([1,0,1,1,0]))
s2 = pred_to_samples(20,torch.FloatTensor([1,0,1,1,0]))
print(s1[0])
print(s2[0])
a = [s1,s2]
a = torch.stack(a, dim =1 )
a.cumsum(dim=1)[0]

In [ ]:
a = (torch.rand((20,8,2))).cumsum(dim=1)
b = (torch.rand((20,8,2))).cumsum(dim=1)
ade_dist(a,b)

In [ ]:
#replace L by H